In [31]:
import pandas as pd
import cv2
from pathlib import Path
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import shutil

In [3]:
df = pd.read_pickle('annot_data.pkl')

In [4]:
pth = Path('./export_data/')

In [5]:
files = os.listdir(pth)

In [6]:
df['file_name']=df['export_name'].str.rsplit('/').str[-1]

In [7]:
df['is_present'] = df['file_name'].isin(files)

In [8]:
coco_info_list = []
image_width=512
image_height=512
for i,r in tqdm(df.iterrows()):
    cords = r['cords']
    x_l = [_[0] for _ in cords]
    y_l = [_[1] for _ in cords]
    x_min = min(x_l)
    x_max = max(x_l)
    y_min = min(y_l)
    y_max = max(y_l)
    x_center = ((x_min + x_max) / 2) / image_width
    y_center = ((y_min + y_max) / 2) / image_height
    width = (x_max - x_min) / image_width
    height = (y_max - y_min) / image_height
    coco_info = [x_center, y_center, width, height]
    coco_info_list.append(coco_info)

875it [00:00, 55024.38it/s]


In [18]:
df['coco_info'] = coco_info_list

In [20]:
_names = df[df['is_present']==True]['file_name'].to_list()

In [21]:
train, _ = train_test_split(_names,train_size=0.7)

In [22]:
val,test = train_test_split(_,train_size=0.666)

In [25]:
for i, names in enumerate([train,val,test]):
    df.loc[df['file_name'].isin(names),'split']=i

In [27]:
name_map = {0:'train',1:'test',2:'val'}

In [42]:
for v in name_map.values():
    _p = os.path.join('./data',v)
    if not os.path.exists(_p):
        os.makedirs(_p)
    _p_im = os.path.join(_p,'images')
    if not os.path.exists(_p_im):
        os.makedirs(_p_im)
    _p_lab = os.path.join(_p,'labels')
    if not os.path.exists(_p_lab):
        os.makedirs(_p_lab)
for i,r in tqdm(df[df['is_present']==True].iterrows()):
    im_path = os.path.join('./data',name_map[r['split']],'images',r['file_name'])
    shutil.copy(r['export_name'],im_path)
    lab_path = os.path.join('./data',name_map[r['split']],'labels',r['file_name'].replace('png','txt'))
    with open(lab_path,'w') as f:
        f.write('0 '+ ' '.join([str(_) for _ in r['coco_info']]))

775it [00:00, 7854.27it/s]
